In [1]:
##Hoping to remove reliance on ArcGIS for ground cover processing

# This tool will not ensure that pixels and boundaries align
# That is the job of the user
import pandas
import numpy as np
import tempfile
from osgeo import gdal, gdalconst
from osgeo import ogr
import glob
import os
import datetime
import calendar
#import matplotlib.pyplot as plt

#requiredStartDate = datetime.date(1990,1,1)
#requiredEndDate = datetime.date(1992,12,31)

#Path to shapefile to use for clip/mask
#pathToShapefile = 'D:/FractionalCover/MackayWhit_Buffer_10km_Albers.shp'
#pathToShapefile = 'D:/FractionalCover/MaryBurnett_Buffer_10km_Albers.shp'

pathToZonesRaster = 'D:/BurnettMary/spatial_RC8/BM_CreatedSubcats.asc'
pathToZonesCategories = 'D:/BurnettMary/spatial_RC8/BM_CreatedSubcats_categories.csv'

tempRasterPath = os.path.join(tempfile.gettempdir(), "resampledData.tif")

#Do one raster only for now
#rasterPathName = 'D:/FractionalCover/GroundCover/lztmre_qld_m198909198911_dixa2.tif'
#Do one raster only for now
#patchPathName = 'D:/FractionalCover/Patches/lztmre_rreef_m198909198911_dj4a2.tif'

#fractionalCoverDir = r'\\nrm02002\EXCHANGE\Fractional Ground Cover\Ground Cover'
#patchDir = r'\\nrm02002\EXCHANGE\Fractional Ground Cover\Ground Cover Patch'
fractionalCoverDir = r'D:\FractionalCover\GroundCover'
patchDir = r'D:\FractionalCover\Patches'

#outputDir = r'D:\FractionalCover\Outputs'

requiredStartDate = datetime.date(1990,1,1)
requiredEndDate = datetime.date(1990,3,31)

#Use these to control the saved files
saveCover = False
#saveCFact = True

extensions = ('*.tif', '*.tiff') # the tuple of file types

availCoverRasters = []
availPatchRasters = []

for theExt in extensions:
    ###
    availCoverRasters.extend(glob.glob(fractionalCoverDir + "/" + theExt))
    availPatchRasters.extend(glob.glob(patchDir + "/" + theExt))

#availCoverRasters = glob.glob(fractionalCoverDir + "/*.tif")
#availPatchRasters = glob.glob(patchDir)

#Final NoData Val: can't use 0, as that is a 'valid' cover val....
ourNoData = 240

#some data storage bits and pieces
allStats = {}
catchIDs = {}
foundStartDates = []


In [2]:
#load the zones raster
zoneRas = gdal.Open(pathToZonesRaster)
zone_GeoTrans = zoneRas.GetGeoTransform()
zonesRasterArray = zoneRas.ReadAsArray()
uniqZones = np.unique(zonesRasterArray)
ZoneNoDataVal = zoneRas.GetRasterBand(1).GetNoDataValue()
#uniqZones.remove(ZoneNoDataVal)

##Place holders
#for zID in uniqZones:
#    allStats[zID] = {}

#zone_GeoTrans

#Open shapefile
#clipPolys = ogr.Open(pathToShapefile)
#lyr = clipPolys.GetLayer(0)
#clipExt = lyr.GetExtent()

#Print feature count
#lyr.GetFeatureCount()

AttributeError: 'numpy.ndarray' object has no attribute 'remove'

In [3]:
index = np.argwhere(uniqZones==ZoneNoDataVal)
uniqZones = np.delete(uniqZones, index)

#Place holders
for zID in uniqZones:
    allStats[zID] = {}


In [4]:
def reProjectAndReSampleRaster(templateRaster, dataRaster):
    dataProj = dataRaster.GetProjection()
    dataRef = dataRaster.GetRasterBand(1)
    referenceProj = templateRaster.GetProjection()
    referenceTrans = templateRaster.GetGeoTransform()
    x = templateRaster.RasterXSize
    y = templateRaster.RasterYSize
    #Keep data type simple for storage space, we know we're dealing with integers here
    #Allowing nearest neighbour resampling
    output = gdal.GetDriverByName("GTiff").Create(tempRasterPath, x, y, 1, dataRef.DataType)
    ###Need a 'No Data Val' that is different to Zero, because that will cause dramas for zero-cover...
    #Can't use -9999!, use 240 as it is higher than any valid data value
    output.GetRasterBand(1).SetNoDataValue(ourNoData)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)
    #gdalconst.GRA_Bilinear
    gdal.ReprojectImage(dataRaster,output,dataProj,referenceProj,gdalconst.GRA_NearestNeighbour)
    

In [5]:
def processCoverFile(fullFileName):
    
    fName = os.path.basename(fullFileName)
    
    beginningIdx = fName.index('_m') + 2
    endingIdx = beginningIdx + 12
    #print("Found at: " + str(beginningIdx))
    #print("Found at: " + fName[beginningIdx:endingIdx])
    
    searchBit = fName[beginningIdx:endingIdx]
    
    theStartDate = datetime.date(int(fName[beginningIdx:beginningIdx+4]),int(fName[beginningIdx+4:beginningIdx+6]),1)
    
    endYear = fName[beginningIdx+6:beginningIdx+10]
    endMonth = fName[beginningIdx+10:beginningIdx+12]
    
    maxDay = calendar.monthrange(int(endYear), int(endMonth))[1]
    
    theEndDate = datetime.date(int(endYear),int(endMonth),maxDay)
    
    if (((theStartDate >= requiredStartDate) and (theStartDate <= requiredEndDate)) or ((theEndDate >= requiredStartDate) and (theEndDate <= requiredEndDate))):
        #Yep, process this fellow
        print("Processing cover for period: " + str(theStartDate) + " to " + str(theEndDate))
        
        alreadyHaveDate = False
        for existSDate in foundStartDates:
            if existSDate == str(theStartDate):
                alreadyHaveDate = True
                break
            
        if alreadyHaveDate == False:
            foundStartDates.append(str(theStartDate))
        
        
        
        #open cover raster
        print("Opening fractional cover: " + fName)
        covRas = gdal.Open(fullFileName)
        covRas_GeoTrans = covRas.GetGeoTransform()
        
        if (covRas_GeoTrans != zone_GeoTrans):
            #Need to reproject/resample
            print("Cover raster resolution info didn't match template, attempted a resample")
            reProjectAndReSampleRaster(zoneRas, covRas)
            #Load resampled raster from temp area
            covRas = gdal.Open(tempRasterPath)
            covRas_GeoTrans = covRas.GetGeoTransform()
            #Do we check????
            if (covRas_GeoTrans == zone_GeoTrans):
                print("Cover raster resolution now matches")
            
        
        #find & open patch raster
        patchFileName = None
        for patchFile in availPatchRasters:
            if(os.path.basename(patchFile).find(searchBit) != -1):
                patchFileName = patchFile
                break
        
        
        print("Opening patch data: " + os.path.basename(patchFileName))
        patchRas = gdal.Open(patchFileName)
        patchRas_GeoTrans = patchRas.GetGeoTransform()
        
        if (patchRas_GeoTrans != zone_GeoTrans):
            print("Patch raster resolution info didn't match template, attempted a resample")
            reProjectAndReSampleRaster(zoneRas, patchRas)
            #Load resampled raster from temp area
            patchRas = gdal.Open(tempRasterPath)
            patchRas_GeoTrans = patchRas.GetGeoTransform()
            if (patchRas_GeoTrans == zone_GeoTrans):
                print("Patch raster resolution now matches")
        
        
        #Should be no need to do clipping now, re-sampling would've done it?
        
        #newRasterInfo = shapeExtent_To_Raster_Box(clipExt, origRas_GeoTrans)
        #patchRasterInfo = shapeExtent_To_Raster_Box(clipExt, origPatchRas.GetGeoTransform())
        
        print("Working...")
        sourceRastArray = covRas.GetRasterBand(1).ReadAsArray()
        patchRastArray = patchRas.GetRasterBand(1).ReadAsArray()

        ##clip = rast[ulY:lrY, ulX:lrX]
        #clipSourceRast = sourceRast[newRasterInfo[8]:newRasterInfo[9],newRasterInfo[6]:newRasterInfo[7]]
        #clipPatchRast = patchRast[patchRasterInfo[8]:patchRasterInfo[9],patchRasterInfo[6]:patchRasterInfo[7]]


        ##Introduce mask of NoDatas
        ##This will include ocean as well as gaps
        ##clipSourceRast = numpy.ma.masked_values(clipSourceRast, origRas.GetRasterBand(1).GetNoDataValue())
        ##clipPatchRast = numpy.ma.masked_values(clipPatchRast, origPatchRas.GetRasterBand(1).GetNoDataValue())
        #clipSourceRast = numpy.ma.masked_where((clipSourceRast < 100) | (clipSourceRast > 200) | (clipSourceRast ==  origRas.GetRasterBand(1).GetNoDataValue()), clipSourceRast)
        #clipPatchRast = numpy.ma.masked_where((clipPatchRast < 100) | (clipPatchRast > 200) | (clipPatchRast == origPatchRas.GetRasterBand(1).GetNoDataValue()), clipPatchRast)
        clipSourceRast = np.ma.masked_where((sourceRastArray < 100) | (sourceRastArray > 200) | (sourceRastArray ==  covRas.GetRasterBand(1).GetNoDataValue()), sourceRastArray)
        clipPatchRast = np.ma.masked_where((patchRastArray < 100) | (patchRastArray > 200) | (patchRastArray == patchRas.GetRasterBand(1).GetNoDataValue()), patchRastArray)

        
        #Free up some memory
        sourceRastArray = None
        patchRastArray = None
        
        clipPatchRast = np.ma.masked_values(clipPatchRast, patchRas.GetRasterBand(1).GetNoDataValue())
        
        insertedPatch = np.ma.masked_values(clipSourceRast.filled(clipPatchRast), patchRas.GetRasterBand(1).GetNoDataValue())
        
        #Need to change data type of array so that we can go above values of 255
        patchedMask = np.array(insertedPatch, dtype='f')
        patchedMask = np.ma.masked_values(patchedMask, patchRas.GetRasterBand(1).GetNoDataValue())
        
        ###Need a 'No Data Val' that is different to Zero, because that will cause dramas for zero-cover...
        #Can't use -9999!, go 240
        patchedMask = patchedMask.filled(ourNoData)
        patchedMask = np.ma.masked_values(patchedMask, ourNoData)
        
        #Important! Use 200.0, not '200', to ensure we retain 'float' data type
        gCov = 200.0-patchedMask
        
        visPatched = 0.00925 * np.power(gCov, 2)

        ##This essential to allow us insert NoDatas in final product... again....
        #visPatched = visPatched.filled(ourNoData)
        #visPatched = numpy.ma.masked_values(visPatched, ourNoData)
        
        ####Now we can do zonal stats......
        ##Just use numpy arrays
        print("Doing zonal stats on " + str(len(uniqZones) + 1) + " unique zones")
        for zID in uniqZones:
            if zID == ZoneNoDataVal:
                print("Ignoring value: " + str(zID))
                continue
                
            
            masked = np.ma.masked_where(np.not_equal(zonesRasterArray, zID), visPatched)
            allStats[zID][str(theStartDate)] = float(masked.mean())
            print("Added to catchID:" + str(zID) + " meanCover: " + str(allStats[zID][str(theStartDate)]))
        
        
        
        #Cfactor....
        #cFactor = numpy.exp(-0.799 - (0.0474 * visPatched) + (0.000449 * numpy.power(visPatched, 2)) - (0.0000052 * numpy.power(visPatched, 3)))
        #cFactor = numpy.ma.masked_values(cFactor, ourNoData)
        
        if (saveCover):
            ###
            outputFile = os.path.join(outputDir, "cov_" + searchBit + ".tif")
            
            print("Saving: " + outputFile)

            #Integer will be fine for cover bands
            #GDT_Byte is 8 bit unsigned (apparently)
            outDS = gdal.GetDriverByName("GTiff").Create(outputFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Byte)
            outDS.SetGeoTransform((newRasterInfo[0], newRasterInfo[2], 0, newRasterInfo[1], 0, newRasterInfo[3]))
            
            #outDS.SetProjection(origRas.GetProjection())
            outBand = outDS.GetRasterBand(1)

            # write the data
            outBand.WriteArray(visPatched)

            # flush data to disk, set the NoData value and calculate stats
            outBand.FlushCache()
            outBand.SetNoDataValue(ourNoData)
            outDS.SetProjection(origRas.GetProjection())
            
            del outDS, outBand

        #if (saveCFact):
        #    ##
        #    outputCFactFile = os.path.join(outputDir, "cf_" + searchBit + ".tif")
        #    
        #    print("Saving: " + outputCFactFile)
        #    
        #    outDSCFact = gdal.GetDriverByName("GTiff").Create(outputCFactFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Float32)
        #    outDSCFact.SetGeoTransform((newRasterInfo[0], newRasterInfo[2], 0, newRasterInfo[1], 0, newRasterInfo[3]))
        #
        #    outDSCFact.SetProjection(origRas.GetProjection())
        #    outBandCFact = outDSCFact.GetRasterBand(1)
        #
        #    outBandCFact.WriteArray(cFactor)
        #    outBandCFact.FlushCache()
        #    outBandCFact.SetNoDataValue(ourNoData)
        #
        #    # georeference the image and set the projection (ROB: don't do GeoTransform, we've already done that)
        #    ##outDs.SetGeoTransform(inDs.GetGeoTransform())
            outDSCFact.SetProjection(origRas.GetProjection())

            #Without this 'del', the output file doesn't get populated with values???
            del outDSCFact, outBandCFact

            
            
        clipPatchRast = None
        clipSourceRast = None
        patchedMask = None
        gCov = None
        visPatched = None
        cFactor = None
        

In [6]:

for theFile in availCoverRasters:
    processCoverFile(theFile)

print("All done")


Processing cover for period: 1989-12-01 to 1990-02-28
Opening fractional cover: lztmre_qld_m198912199002_dixa2.tif
Cover raster resolution info didn't match template, attempted a resample
Cover raster resolution now matches
Opening patch data: lztmre_rreef_m198912199002_dj4a2.tif
Patch raster resolution info didn't match template, attempted a resample
Patch raster resolution now matches
Working...
Doing zonal stats on 683 unique zones
Added to catchID:1 meanCover: 60.25048008458119
Added to catchID:2 meanCover: 66.61321532626023
Added to catchID:3 meanCover: 61.87172704834562
Added to catchID:4 meanCover: 69.21777847901546
Added to catchID:5 meanCover: 68.80333353390698
Added to catchID:6 meanCover: 72.26712556997545
Added to catchID:7 meanCover: 79.85036804430038
Added to catchID:8 meanCover: 75.49598943679352
Added to catchID:9 meanCover: 78.4787157060156
Added to catchID:10 meanCover: 52.53414575289575
Added to catchID:11 meanCover: 82.77604399820937
Added to catchID:12 meanCover: 7

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:135: UserWarning: Warning: converting a masked element to nan.


Added to catchID:174 meanCover: nan
Added to catchID:175 meanCover: 74.22018725361367
Added to catchID:176 meanCover: 73.16262577741821
Added to catchID:177 meanCover: 76.78654694715239
Added to catchID:178 meanCover: 66.67471741928264
Added to catchID:179 meanCover: 55.09816886716964
Added to catchID:180 meanCover: 77.46810047006853
Added to catchID:181 meanCover: 78.5489104892117
Added to catchID:182 meanCover: 71.59557262711007
Added to catchID:183 meanCover: 63.12362838332114
Added to catchID:184 meanCover: 73.71974035608308
Added to catchID:185 meanCover: 60.26582786417463
Added to catchID:186 meanCover: 75.15316964285714
Added to catchID:187 meanCover: 70.72040024405125
Added to catchID:188 meanCover: 73.57752246615857
Added to catchID:189 meanCover: 75.6212964348043
Added to catchID:190 meanCover: 60.18376218026797
Added to catchID:191 meanCover: 76.32761402753873
Added to catchID:192 meanCover: 71.49777050548246
Added to catchID:193 meanCover: 63.3099349266052
Added to catchID:

In [7]:
#Now to get at the data...
print(foundStartDates)

['1989-12-01', '1990-03-01']


In [11]:
dFrame = pandas.DataFrame()
dFrame['sDates'] = foundStartDates.sort()

sTd = foundStartDates.sort()

for unZID in uniqZones:
    for eachSDate in sTd:
        dFrame[str(unZID)] = allStats[zID][eachSDate]

#for eachSDate in foundStartDates.sorted():
#    for unZID in uniqZones:
#        dFrame[str(unZID)] = allStats[zID][eachSDate]

TypeError: 'NoneType' object is not iterable